In [54]:
import os
from pathlib import Path

import aiobotocore
import pandas as pd
from dotenv import load_dotenv
from numpy import dtype
from s3fs import S3FileSystem
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split


In [55]:
load_dotenv()
[os.getenv("AWS_PROFILE"), os.getenv("TRAINING_DIR")]

['Prod', None]

In [56]:
tmpdir = Path.cwd().parent / "tmp"
s3 = S3FileSystem(session=aiobotocore.session.AioSession(profile=os.getenv("AWS_PROFILE")))
bucket_root = f"s3://{os.getenv("BUCKET_NAME")}/ny_taxi_trip_prediction"

if os.getenv("TRAINING_DIR"):
    training_base_dir = os.getenv("TRAINING_DIR")
else:
    training_base_dir = s3.read_text(f"{bucket_root}/current")

training_root = f"{bucket_root}/training/{training_base_dir}"

train_path = tmpdir / "train.parquet"
if not train_path.is_file():
    s3.get_file(training_root + "/train.parquet", train_path)
df_train_val_all = pd.read_parquet(train_path)

test_path = tmpdir / "test.parquet"
if not test_path.is_file():
    s3.get_file(training_root + "/test.parquet", test_path)
df_test_all = pd.read_parquet(test_path)


In [57]:
assert df_train_val_all['PULocationID'].dtypes is dtype("int32")
assert df_test_all['PULocationID'].dtypes is dtype("int32")

In [58]:
df_train_val_all.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-07-01 00:34:56,2024-07-01 00:46:49,1.0,3.20,1.0,N,140,79,1,15.6,3.50,0.5,3.50,0.00,1.0,24.10,2.5,0.00
1,2,2024-06-30 23:48:58,2024-07-01 00:28:04,1.0,19.48,2.0,N,132,113,2,70.0,0.00,0.5,0.00,0.00,1.0,75.75,2.5,1.75
2,2,2024-07-01 00:23:18,2024-07-01 00:29:51,1.0,1.18,1.0,N,237,145,1,8.6,1.00,0.5,2.72,0.00,1.0,16.32,2.5,0.00
3,1,2024-07-01 00:10:33,2024-07-01 00:27:31,0.0,9.10,1.0,N,138,164,1,36.6,10.25,0.5,12.05,0.00,1.0,60.40,2.5,1.75
4,1,2024-07-01 00:07:55,2024-07-01 00:34:34,1.0,17.70,2.0,N,132,263,1,70.0,1.75,0.5,10.00,6.94,1.0,90.19,0.0,1.75


In [59]:
df_train_val_all[df_train_val_all.index.duplicated()]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee


In [60]:
def preprocess(df: pd.DataFrame, dict_vec: DictVectorizer | None = None, debug_output=False) -> [any, any,
                                                                                                 DictVectorizer]:
    """ Returns x, y, and dict vectorizer. If dict vectorizer was supplied in arguments, does not fit, only transforms.
    """
    dict_vec_fit = False
    if dict_vec is None:
        dict_vec = DictVectorizer()
        dict_vec_fit = True

    tmpdf = pd.DataFrame(index=df.index)
    tmpdf["duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).apply(
        lambda timediff: timediff.total_seconds())
    tmpdf = tmpdf[(tmpdf["duration_min"] >= 1) & (tmpdf["duration_min"] <= 60)]
    tmpdf["loc_id"] = df["PULocationID"].astype(str) + "-" + df["DOLocationID"].astype(str)
    tmpdf["trip_distance"] = df["trip_distance"]
    if debug_output:
        print("tmpdf")
        display(tmpdf)

    x_dicts = tmpdf[["loc_id", "trip_distance"]].to_dict(orient="records")
    if debug_output:
        print("x_dicts")
        display(x_dicts)

    if dict_vec_fit:
        x = dict_vec.fit_transform(x_dicts)
    else:
        x = dict_vec.transform(x_dicts)
    y = tmpdf["duration_min"].values

    if debug_output:
        print("x")
        display(x)
        print("y")
        display(y)

    return x, y, dict_vec

In [61]:
# Take first 80% for training set, last 20% for validation set. Order DOES matter because we want to use later data for validation as it is in theory closer to reality
xy_train_all, xy_val_all = train_test_split(df_train_val_all, test_size=0.2, shuffle=False)
display(xy_train_all.head())
display(xy_val_all.head())

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-07-01 00:34:56,2024-07-01 00:46:49,1.0,3.20,1.0,N,140,79,1,15.6,3.50,0.5,3.50,0.00,1.0,24.10,2.5,0.00
1,2,2024-06-30 23:48:58,2024-07-01 00:28:04,1.0,19.48,2.0,N,132,113,2,70.0,0.00,0.5,0.00,0.00,1.0,75.75,2.5,1.75
2,2,2024-07-01 00:23:18,2024-07-01 00:29:51,1.0,1.18,1.0,N,237,145,1,8.6,1.00,0.5,2.72,0.00,1.0,16.32,2.5,0.00
3,1,2024-07-01 00:10:33,2024-07-01 00:27:31,0.0,9.10,1.0,N,138,164,1,36.6,10.25,0.5,12.05,0.00,1.0,60.40,2.5,1.75
4,1,2024-07-01 00:07:55,2024-07-01 00:34:34,1.0,17.70,2.0,N,132,263,1,70.0,1.75,0.5,10.00,6.94,1.0,90.19,0.0,1.75


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
5292876,2,2024-06-21 21:23:04,2024-06-21 21:27:28,1.0,0.28,1.0,N,100,48,1,5.8,1.0,0.5,0.00,0.0,1.0,10.80,2.5,0.0
5292877,2,2024-06-21 21:44:07,2024-06-21 22:00:44,2.0,1.14,1.0,N,48,233,1,14.9,1.0,0.5,3.98,0.0,1.0,23.88,2.5,0.0
5292878,2,2024-06-21 21:46:01,2024-06-21 21:50:12,2.0,0.78,1.0,N,161,237,2,6.5,1.0,0.5,0.00,0.0,1.0,11.50,2.5,0.0
5292879,2,2024-06-21 21:27:12,2024-06-21 21:40:29,1.0,2.33,1.0,N,236,161,1,14.9,1.0,0.5,3.98,0.0,1.0,23.88,2.5,0.0
5292880,2,2024-06-21 21:57:42,2024-06-21 22:12:21,4.0,2.42,1.0,N,162,148,2,14.9,1.0,0.5,0.00,0.0,1.0,19.90,2.5,0.0


In [62]:
x_train, y_train, dv = preprocess(xy_train_all, debug_output=False)

In [63]:
x_val, y_val, _ = preprocess(xy_val_all, dv, debug_output=False)

In [64]:
x_test, y_test, _ = preprocess(df_test_all, dv, debug_output=False)

In [65]:
model = RandomForestRegressor(max_depth=10, random_state=0)
model.fit(x_train, y_train)
y_pred_val = model.predict(x_val)
y_pred_test = model.predict(x_test)

print(f"RMSE of validation set: {float(root_mean_squared_error(y_val, y_pred_val))}")
print(f"RMSE of test set: {float(root_mean_squared_error(y_test, y_pred_test))}")

RMSE of validation set: 13.783918857799998
RMSE of test set: 13.382332322020392


In [66]:
print("Pick up location IDs")
display(xy_train_all["PULocationID"].sort_values().unique())
print("Drop off location IDs")
display(xy_train_all["DOLocationID"].sort_values().unique())

Pick up location IDs


array([  1,   2,   3,   4,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 105, 106, 107,
       108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 18

Drop off location IDs


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 106, 107,
       108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 18